## Simple choice model
Description and examples: https://www.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-discrete-choice_models.html#the-basic-model

In [ ]:
# ------------------------------------------------------------------
# Load Libraries & Data
# ------------------------------------------------------------------
import os
os.environ['PYTENSOR_FLAGS'] = 'cxx=C:/msys64/ucrt64/bin/g++.exe,optimizer=fast_compile'

import re
import pymc as pm
import arviz as az

import pandas as pd
import numpy as np

print(os.getcwd())

print(pm.__name__, pm)

In [ ]:
# Version 1: prepare dummies without baseline in data for model
# load preprocessed data
df = pd.read_csv('../results/conjoint_df.csv', index_col=0)

# experimental variables and defining baseline values
attr_cols = ['costs', 'exemptions', 'benefits']
baselines = {'costs': 'All people pay the same amount', 'exemptions': 'No groups exempted from costs', 'benefits': 'Equal protection levels for all municipalities'}

#  reorder experimental columns making it Categorical starting with chosen baseline
for a in attr_cols:
    unique_levels = df[a].dropna().unique()
    cats = [baselines[a]] + [lvl for lvl in unique_levels if lvl != baselines[a]]
    df[a] = pd.Categorical(df[a], categories=cats)

# create dummies without baseline attribute level
df_num = pd.get_dummies(df[attr_cols], drop_first=True)

In [ ]:
# Version 2: prepare dummies including baseline
# TODO: asking if rewriting code like this is fine or not, obviously with adding credits, if i want to cite sth then i can also do that later using ccs-preferences as kristiina finishes this 
# load preprocessed data
df = pd.read_csv('../results/conjoint_df.csv', index_col=0)

# experimental variables and defining baseline values
attr_cols = ['costs', 'exemptions', 'benefits']
baselines = {'costs': 'All people pay the same amount', 'exemptions': 'No groups exempted from costs', 'benefits': 'Equal protection levels for all municipalities'}

#  reorder experimental columns making it Categorical starting with chosen baseline
for a in attr_cols:
    unique_levels = df[a].dropna().unique()
    cats = [baselines[a]] + [lvl for lvl in unique_levels if lvl != baselines[a]]
    df[a] = pd.Categorical(df[a], categories=cats)

# create dummies without baseline attribute level
df_num = pd.get_dummies(df[attr_cols], drop_first=False)

# Ordne die Spalten so, dass die Baselines zuerst kommen
col_order = []
for a in attr_cols:
    # Filtere Spalten, die zur aktuellen Variablen gehören
    attr_vals = [col for col in df_num.columns if col.startswith(a)]
    # Füge zuerst die Baseline hinzu, dann die restlichen Level
    base_val = f'{a}_{baselines[a]}'
    col_order.append(base_val)
    col_order.extend([val for val in attr_vals if val != base_val])

# Ordne die Spalten neu und entferne Duplikate (falls vorhanden)
df_num = df_num[col_order]
df_num = df_num.loc[:, ~df_num.columns.duplicated()]


# check the result
print('Dummy-Variablen:')
print(df_num.head())


In [ ]:
# build task id
task_key = ['respondent_id', 'nh_event', 'task']
df['task_id'] = pd.factorize(df[task_key].apply(tuple, axis=1))[0]

# sort, so both, left and right option have same order
df = df.sort_values(['task_id', 'option']).copy()

# define right and left options
left_option  = 1
right_option = 2
left_mask  = df['option'] == left_option
right_mask = df['option'] == right_option

# check: per taks two rows with only one of them being chosen
counts = df.groupby('task_id').size()
assert (counts == 2).all(), 'Es gibt Tasks, die nicht genau 2 Alternativen haben.'
chosen_sum = df.groupby('task_id')['chosen'].sum()
assert (chosen_sum == 1).all(), 'Es gibt Tasks, bei denen chosen nicht genau einmal 1 ist.'

# X-matrices
X_left  = df_num.loc[left_mask].to_numpy()
X_right = df_num.loc[right_mask].to_numpy()

# observing, which option was chosen? was it left?
y_left = df.loc[left_mask, 'chosen'].to_numpy().astype(int)

# framing: nh_event per task (0/1), from left chosen 
f = df.loc[left_mask, 'nh_event'].to_numpy().astype(int)

n_tasks = X_left.shape[0]

coords = {
    'level': df_num.columns.values,
    'task': np.arange(n_tasks)
}

In [ ]:
with pm.Model(coords = coords) as simple_model:
    # TODO change sigma and check what fits best, play with it, attribute levels: sigma 2
    # TODO: check beta values against veronikas 
    # sigma of latent constructs is expected to be smaller, so more than sigma 1
    # sigma 2 was tested but did not lead to any other results when not including the baseline directly in the dummies for the model
    beta = pm.Normal('beta', mu=0, sigma = 1, dims = 'level')
    
    # data
    Xl = pm.Data('X_left', X_left, dims=('task', 'level'))
    Xr = pm.Data('X_right', X_right, dims=('task', 'level'))
    y = pm.Data('y_left', y_left, dims='task')
    
    u_left = pm.Deterministic('u_left', pm.math.sum(Xl * beta, axis = 1), dims='task')
    u_right = pm.Deterministic('u_right', pm.math.sum(Xr * beta, axis = 1), dims='task')
    
    p_left = pm.Deterministic('p_left', pm.math.sigmoid(u_left - u_right), dims='task') # not necessarily needed
    logit_p_left = pm.Deterministic('logit_p_left', u_left - u_right, dims='task')
    
    choice = pm.Bernoulli('choice', logit_p=logit_p_left, observed=y, dims='task')




In [ ]:
# model test
with simple_model:
    approx = pm.fit(100, method='advi')
    # idata_vi = approx.sample(500, return_inferencedata=True)



In [ ]:
# model test better approximation
with simple_model:
    # idata_mcmc_with_base = pm.sample(draws=500, tune=500, chains=4, cores = 4, target_accept=0.9, progressbar=True)
    idata_mcmc = pm.sample(draws=500, tune=500, chains=4, cores = 4, target_accept=0.9, progressbar=True)

In [ ]:
az.summary(idata_mcmc, var_names=['beta'])
az.plot_trace(idata_mcmc, var_names=['beta'])
az.plot_forest(idata_mcmc, var_names=['beta'], combined=True)

levels = list(idata_mcmc.posterior['beta'].coords['level'].values)

groups = {
    'Costs':      [lv for lv in levels if lv.startswith('costs_')],
    'Benefits':   [lv for lv in levels if lv.startswith('benefits_')],
    'Exemptions': [lv for lv in levels if lv.startswith('exemptions_')],
}

axes = az.plot_forest(
    [idata_mcmc, idata_mcmc, idata_mcmc],
    var_names=['beta', 'beta', 'beta'],
    coords=[{'level': groups['Costs']},
            {'level': groups['Benefits']},
            {'level': groups['Exemptions']}],
    model_names=['Costs', 'Benefits', 'Exemptions'],
    combined=True,
    legend=True
)


In [ ]:
az.summary(idata_mcmc_with_base, var_names=['beta'])
az.plot_trace(idata_mcmc_with_base, var_names=['beta'])
az.plot_forest(idata_mcmc_with_base, var_names=['beta'], combined=True)

levels = list(idata_mcmc_with_base.posterior['beta'].coords['level'].values)

groups = {
    'Costs':      [lv for lv in levels if lv.startswith('costs_')],
    'Benefits':   [lv for lv in levels if lv.startswith('benefits_')],
    'Exemptions': [lv for lv in levels if lv.startswith('exemptions_')],
}

axes = az.plot_forest(
    [idata_mcmc_with_base, idata_mcmc_with_base, idata_mcmc_with_base],
    var_names=['beta', 'beta', 'beta'],
    coords=[{'level': groups['Costs']},
            {'level': groups['Benefits']},
            {'level': groups['Exemptions']}],
    model_names=['Costs', 'Benefits', 'Exemptions'],
    combined=True,
    legend=True
)


In [ ]:
idata_mcmc

In [ ]:
df = az.summary(idata_mcmc, var_names=['beta'], hdi_prob=0.95).reset_index()
print(df)

df['hdi_2.5%']

In [ ]:
summ = az.summary(idata_mcmc, var_names=['beta'], hdi_prob=0.95).reset_index()

summ['level_raw'] = summ['index'].str.extract(r'beta\[(.*)\]')[0]
summ['group'] = summ['level_raw'].str.extract(r'^(costs|benefits|exemptions)_')[0]
summ['label'] = summ['level_raw'].str.replace(r'^(costs|benefits|exemptions)_', '', regex=True)
summ['group'] = pd.Categorical(summ['group'], ['costs', 'benefits', 'exemptions'], ordered=True)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

groups = ['costs', 'benefits', 'exemptions']
titles = {'costs':'Costs', 'benefits':'Benefits', 'exemptions':'Exemptions'}
color_map = {'costs': 'tab:blue', 'benefits': 'tab:orange', 'exemptions': 'tab:green',}

# common coherent x limits for clear visualization
xmin = summ['hdi_2.5%'].min() - 0.1
xmax = summ['hdi_97.5%'].max() + 0.1

# Height of a subplot depending on number of attribute-levels
heights = [max(1.4, 0.55 * len(summ[summ['group'] == g])) for g in groups]

fig, axes = plt.subplots(
    nrows=3, ncols=1, sharex=True,
    figsize=(10, sum(heights)),
    gridspec_kw={'height_ratios': heights}
)

for ax, g in zip(axes, groups):
    d = summ[summ['group'] == g].copy()

    # sorting with increasing mean
    d = d.sort_values('mean')

    y = np.arange(len(d))[::-1]
    c = color_map[g]

    ax.hlines(y, d['hdi_2.5%'], d['hdi_97.5%'], linewidth=2, color=c)
    ax.plot(d['mean'], y, 'o', color=c)

    ax.axvline(0, ls='--', lw=1, color='gray')
    ax.set_yticks(y)
    ax.set_yticklabels(d['label'])
    ax.set_title(titles[g], loc='left', fontweight='bold')
    ax.grid(axis='x', alpha=0.2)
    ax.set_xlim(xmin, xmax)

axes[-1].set_xlabel('Posterior mean and 95% credible interval')
plt.tight_layout()
plt.show()


In [ ]:
summ = az.summary(idata_mcmc, var_names=['beta'], hdi_prob=0.95).reset_index()

summ['level_raw'] = summ['index'].str.extract(r'beta\[(.*)\]')[0]
summ['group'] = summ['level_raw'].str.extract(r'^(costs|benefits|exemptions)_')[0]
summ['label'] = summ['level_raw'].str.replace(r'^(costs|benefits|exemptions)_', '', regex=True)
summ['group'] = pd.Categorical(summ['group'], ['costs', 'benefits', 'exemptions'], ordered=True)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

groups = ['costs', 'benefits', 'exemptions']
titles = {'costs':'Costs', 'benefits':'Benefits', 'exemptions':'Exemptions'}

baselines = {
    'costs':      'All people pay the same amount',
    'benefits':   'Equal protection levels for all municipalities',
    'exemptions': 'No groups exempted from costs',
}

baseline_rows = []
for g, name in baselines.items():
    baseline_rows.append({
        'index': f'beta[{g}_{name}]',
        'mean': 0.0,
        'sd': 0.0,
        'hdi_2.5%': 0.0,
        'hdi_97.5%': 0.0,
        'group': g,
        'label': name,
    })
summ = pd.concat([summ, pd.DataFrame(baseline_rows)], ignore_index=True)


# Farben pro Gruppe (du kannst die Strings ändern)
color_map = {
    'costs': 'tab:blue',
    'benefits': 'tab:orange',
    'exemptions': 'tab:green',
}

# gemeinsame x-limits (optional, aber sieht meist besser aus)
xmin = summ['hdi_2.5%'].min() - 0.1
xmax = summ['hdi_97.5%'].max() + 0.1

# Höhe pro Panel abhängig von Anzahl Levels
heights = [max(1.4, 0.55 * len(summ[summ['group'] == g])) for g in groups]

fig, axes = plt.subplots(
    nrows=3, ncols=1, sharex=True,
    figsize=(10, sum(heights)),
    gridspec_kw={'height_ratios': heights}
)

for ax, g in zip(axes, groups):
    d = summ[summ['group'] == g].copy()

    # optional: sortieren nach mean (wenn du Originalreihenfolge willst, löschen)
    d = d.sort_values('mean')

    y = np.arange(len(d))[::-1]
    c = color_map[g]

    xerr = np.vstack([d['mean'] - d['hdi_2.5%'], d['hdi_97.5%'] - d['mean']])  # (2, n)
    ax.errorbar(
        d['mean'], y,
        xerr=xerr,
        fmt='o',
        color=c,
        ecolor=c,
        elinewidth=2,
        capsize=4,      # <<< Länge der “Balken” an den Enden
        capthick=2
    )

    ax.axvline(0, ls='--', lw=1, color='gray')
    ax.set_yticks(y)
    ax.set_yticklabels(d['label'])
    ax.set_title(titles[g], loc='left', fontweight='bold')
    ax.grid(axis='x', alpha=0.2)
    ax.set_xlim(xmin, xmax)

axes[-1].set_xlabel('Posterior mean and 95% credible interval')
plt.tight_layout()
plt.show()

# vgl 

In [ ]:
# model run
with simple_model:
    idata = pm.sample(
        draws = 1000, 
        tune = 500, 
        chains = 4,
        cores = 4, 
        random_seed = 42, 
        return_inferencedata = True, 
        target_accept = 0.9, 
        progressbar=True
    )